In [1]:
import tensorflow as tf
import json
import base64
import numpy as np
import requests


def prepare_example(text):
    # Define the features of your data
    feature = {
        "text": tf.train.Feature(bytes_list=tf.train.BytesList(value=[text.encode()]))
    }

    # Create an Example
    example = tf.train.Example(features=tf.train.Features(feature=feature))

    # Serialize the Example to a string
    example_str = example.SerializeToString()

    # Prepare the JSON data
    json_data = {
        "signature_name": "serving_default",
        "instances": [{"b64": base64.b64encode(example_str).decode()}]
    }

    return json.dumps(json_data)

json_data = prepare_example('i gave up my internship with the dmrg and am feeling distraught')

endpoint = "http://localhost:8080/v1/models/emotion-detection-model:predict"
response = requests.post(endpoint, data=json_data)

if response.status_code == 200:
    prediction = response.json().get("predictions")
    if prediction:
        prediction_class = np.argmax(prediction)  # Ambil kelas dengan nilai probabilitas tertinggi
        # Konversi kelas menjadi label sesuai dengan informasi kelas yang diberikan
        labels = {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}
        predicted_label = labels.get(prediction_class, "Unknown")
        result = f"hasil prediksi: {predicted_label}"
    else:
        result = "Error: No predictions found in response."
else:
    result = "Error: Model prediction failed."

print(result)


hasil prediksi: fear
